In [1]:
import matplotlib.pyplot as plt
import pickle
from os import walk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

path = "./UTKFace"

In [4]:
filenames = next(walk(path), (None, None, []))[2]
dataset = []
for f in filenames:
  age = f.split("_")[0]
  dataset.append((plt.imread("./UTKFace/"+f), age))

#Serializzazione
with open("dataset1.pickle", "wb") as outfile:
 pickle.dump(pd.DataFrame(dataset, columns =["x", "y"]), outfile)

In [2]:
with open("dataset.pickle", "rb") as infile:
    df_ds = pickle.load(infile)
print("Oggetto Ricostruito")

Oggetto Ricostruito


In [3]:
x_train_full, x_test_full, y_train_full, y_test = train_test_split(df_ds.x, df_ds.y, test_size = 0.2, random_state=1)
x_valid, x_train = x_train_full[:5000] / 255., x_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
x_test = x_test_full / 255.

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
print(x_train.head())

In [16]:
alex = tf.keras.models.Sequential([
            tf.keras.layers.Resizing(227,227),
            data_augmentation,
            tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4,
                                   activation='relu'),
            keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same',
                                   activation='relu'),
            keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                                   activation='relu'),
            keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same',
                                   activation='relu'),
            keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same',
                                   activation='relu'),
            keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(117, activation="softmax")])


alex.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [5]:
x = []
for i in x_train:
  x.append(tf.convert_to_tensor(i, np.float32))

xv = []
for i in x_valid:
  xv.append(tf.convert_to_tensor(i, np.float32))


In [7]:
y = tf.convert_to_tensor(y_train, np.float32)
yv = tf.convert_to_tensor(y_valid, np.float32)

In [9]:
#Serializzazione tensori
yv = tf.stack(yv)
with open("y_valid.pickle", "wb") as outfile:
 pickle.dump(yv, outfile)

y = tf.stack(y)
with open("y.pickle", "wb") as outfile:
 pickle.dump(y, outfile)

xv = tf.stack(xv)
with open("x_valid.pickle", "wb") as outfile:
 pickle.dump(xv, outfile)

x = tf.stack(x)
with open("x.pickle", "wb") as outfile:
 pickle.dump(x, outfile)

In [17]:
with open("y_valid.pickle", "rb") as infile:
    yv = pickle.load(infile)
with open("y.pickle", "rb") as infile:
    y = pickle.load(infile)
with open("x_valid.pickle", "rb") as infile:
    xv = pickle.load(infile)
with open("x.pickle", "rb") as infile:
    x = pickle.load(infile)
historyA = alex.fit(x, y, epochs=20, validation_data=(xv, yv))

MemoryError: Unable to allocate 6.24 GiB for an array with shape (13966, 200, 200, 3) and data type float32